# AUTH API Utils

In [ ]:
import os
from sqlalchemy import desc
import datedelta
import base64
import uuid

In [ ]:
AUTH_URL = os.getenv('AUTH_URL')
ACCOUNT_SVC_AUTH_URL = os.getenv('ACCOUNT_SVC_AUTH_URL')
ACCOUNT_SVC_CLIENT_ID = os.getenv('ACCOUNT_SVC_CLIENT_ID')
ACCOUNT_SVC_CLIENT_SECRET = os.getenv('ACCOUNT_SVC_CLIENT_SECRET')
ACCOUNT_SVC_AFFILIATE_URL = os.getenv('ACCOUNT_SVC_AFFILIATE_URL')
ACCOUNT_SVC_ENTITY_URL = os.getenv('ACCOUNT_SVC_ENTITY_URL')

In [ ]:
def get_auth_token(verify_ssl = True):
    payload = "grant_type=client_credentials"
    basic_hash = base64.b64encode(f'{ACCOUNT_SVC_CLIENT_ID}:{ACCOUNT_SVC_CLIENT_SECRET}'.encode()).decode()
    basic_hash
    headers = {
      'Content-Type': 'application/x-www-form-urlencoded',
      'Authorization': f'Basic {basic_hash}'
    }

    response = requests.request("POST", ACCOUNT_SVC_AUTH_URL, headers=headers, data=payload, verify=verify_ssl)

    auth_token = response.json()['access_token']
    # auth_token
    assert auth_token
    return auth_token

In [ ]:
def affiliate_nr_to_account(auth_token: str, nr_num: str, email: str, account_id: int, verify_ssl = True):
    url = ACCOUNT_SVC_AFFILIATE_URL.format(account_id=account_id)
    url = f'{url}?newBusiness=true'

    headers = {
      'Authorization': f'Bearer {auth_token}'
    }

    payload = {
      "businessIdentifier": nr_num,
      "phone": "",
      "email": email
    }

    r = requests.post(url, headers=headers, json=payload, verify=verify_ssl)
    return r

In [1]:
def affiliate_nr_to_entity(auth_token: str, nr_num: str, nr_name: str, verify_ssl = True):
    url = f'{ACCOUNT_SVC_ENTITY_URL}/{nr_num}'

    headers = {
      'Authorization': f'Bearer {auth_token}'
    }

    payload = {
      "businessIdentifier": nr_num,
      "name": nr_name
    }

    r = requests.patch(url, headers=headers, json=payload, verify=verify_ssl)    
    return r    

In [1]:
def get_identifier_from_affiliations(auth_token: str, account_id: int, nr_name: str, verify_ssl = True):
    url = ACCOUNT_SVC_AFFILIATE_URL.format(account_id=account_id)
    
    headers = {
      'Authorization': f'Bearer {auth_token}'
    }

    r = requests.get(url, headers=headers, verify=verify_ssl)
    r.status_code, r.text
    response_dict = json.loads(r.text)
    entities = response_dict['entities']

    matching_entities = list(filter(lambda x: x['name'] == approved_nr_name, entities))
    # print(f'matching_entities: {matching_entities}, matches: {len(matching_entities)}')
    assert len(matching_entities) == 1
    identifier = matching_entities[0]['businessIdentifier']
    return identifier

In [ ]:
def get_account_by_affiliated_identifier(auth_token: str, identifier: str, verify_ssl = True):
    url = f'{AUTH_URL}/orgs?affiliation={identifier}'
    
    headers = {
      'Authorization': f'Bearer {auth_token}'
    }

    r = requests.get(url, headers=headers, verify=verify_ssl)
    return r